<a href="https://colab.research.google.com/github/cruz-marco/pyspark_course/blob/main/pyspark_APPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = '/content/spark-3.2.3-bin-hadoop3.2'

!pip install -q findspark

import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

# Criando APPS

In [41]:
%%writefile aplicativo.py
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

if __name__ == '__main__':
  spark = SparkSession.builder.appName('Exemplo').getOrCreate()

  arqschema = ("id INT, nome STRING, status STRING, cidade STRING, vendas INT,"
              " data TIMESTAMP")

  despachantes = spark.read.load(("/content/drive/MyDrive/Datasets/pyspark_cou"
                            "rse/despachantes.csv"), header=False, format="csv", 
                            schema=arqschema)

  calculo = despachantes.select(f.year(f.col('data')).alias('anos'), f.col('id'))\
            .groupBy('anos')\
            .agg(f.count('id').alias('count'))\
            .orderBy(f.col('count').desc())

  calculo.write.format('console').save()
  spark.stop()

Writing aplicativo.py


In [42]:
%%writefile aplicativo2.py
import sys
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

if __name__ == '__main__':
  spark = SparkSession.builder.appName('Exemplo').getOrCreate()

  arqschema = ("id INT, nome STRING, status STRING, cidade STRING, vendas INT,"
              " data TIMESTAMP")

  despachantes = spark.read.load(sys.argv[1], header=False, format="csv", 
                                schema=arqschema)

  calculo = despachantes.select(f.year(f.col('data')).alias('anos'), f.col('id'))\
            .groupBy('anos')\
            .agg(f.count('id').alias('count'))\
            .orderBy(f.col('count').desc())

  calculo.write.format('console').save()
  spark.stop()

Writing aplicativo2.py


In [43]:
%%writefile aplicativo3.py
import sys, getopt
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

if __name__ == '__main__':
  spark = SparkSession.builder.appName('Exemplo').getOrCreate()

  opts, args = getopt.getopt(sys.argv[1:], 't:i:o')
  formato, infile, outdir = '', '', ''

  for opt, arg in opts:
    if opt == '-t':
      formato = arg
    elif opt == '-i':
      infile = arg
    elif opt == '-o':
      outdir = arg

  
  data = spark.read.csv(infile, header=False, inferSchema=True)

  data.write.format(formato).save(outdir)

  spark.stop()

Writing aplicativo3.py
